In [2]:

import os
import sys
import timeit
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

cwd = os.getcwd()
print(cwd)

from BF_main import read_data_and_configs, run_beamformer, run_image_processing
from env_data_op import env_data_get, env_data_compress, env_qbp_filter,env_data_interpolate
from img_data_op import image_conversion, image_polar,image_crop, image_plot_linear, image_plot_curvelinear, image_interpolate, image_LUT, image_contrast, image_sharpness

C:\code\signal_processing_dev\dstMain_reference_model


In [3]:
print("grabbing data and config")

Tim = True

if not Tim:

    # path and presets
    config_dir = 'C:\\Codes\\github\\signal_processing_dev\\SK_reference_model\\presets'#
    file_path = 'C:\\Users\\dst01\\OneDrive - deepsightinc.com\\work\\Projects\\Mixed Array Transducer\\20241016 harmonic imaging\\pic_mrf2-10_3.05mhz_2cyc_20hv_wfm3_inv'
    file_name = "_PIC_b_mode_0_raw_rf"

    probe_dir = 'DL3-14'
    config_path = os.path.join(config_dir,probe_dir)

    preset_name = 'DL3-14_PIC'

else:
    data_dir =  'Challenge_sets/L7-4_General'
    config_path = os.path.join(data_dir, 'presets/scratch')

    raw_rf_dir = os.path.join(data_dir, 'data')

    file_name = 'L7-4_General_b_mode_0_raw_rf.npy'
    raw_rf_path = os.path.join(raw_rf_dir, file_name)

grabbing data and config


In [4]:
rf_data, xdc, field, param = read_data_and_configs(raw_rf_path, config_path)

loading preset Challenge_sets/L7-4_General\presets/scratch


In [5]:
recon_image = run_beamformer(rf_data, xdc, field, param)

processing reconstruction
precomputing geometry...
precomputing field...
performing delay and sum...
running GPU beamformer
beamforming took 34.04718639999919 seconds 


In [6]:
num_samples = rf_data.shape[1]

In [7]:
scale = 7
run_image_processing(recon_image, xdc, field, param, num_samples, scale = scale)

getting envelope
plotting for display


AttributeError: 'NoneType' object has no attribute 'imshow'

In [ ]:
# sum the data through transmits
recon_image_sum = np.sum(recon_image,axis = 1)/2**15 

print("getting envelope") 
# get envelop and compress
recon_image_sum = env_qbp_filter(recon_image_sum, filter_kernel=np.array(param["filter_kernel"]))
recon_image_sum = env_data_interpolate(recon_image_sum,interp_factor = 2)

In [ ]:
# apodize it to prevent hilbert transform artfact
recon_image_sum [:,-4:]=0; recon_image_sum [:,:4]=0

In [ ]:


# recon_US_img_data = env_qbp_filter(recon_US_img_data, filter_kernal=bf_config["line_processor"]["qbp_arms_2"]["filter_kernel"])
recon_US_img_data = env_data_get(recon_image_sum, median_filter_size=1)+1e-12
recon_US_img_dB = env_data_compress(recon_US_img_data,pow=param["filter_power"], value_reject = 0.0)

In [ ]:

# %%
print("plotting for display") 
US_img = image_conversion(recon_US_img_dB/7,Resize_1=1,Resize_2=1) # convert to uint8
US_img = image_crop(US_img, img_length=field["axial_size"]*rf_data.shape[1], linelength = field["line_length"]) # crop basedon available signal length

# adding image manipulation for better display
# US_img = image_interpolate(US_img,interp_channel=4,interp_depth=1) # interpolate the image
# US_img = image_LUT(US_img,LUT = np.linspace(0,255,256).astype('uint8') ) # use LUT
# US_img = image_contrast(US_img, brightness = 1, contrast = 1)
# US_img = image_sharpness(US_img, kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]))

if xdc["type"] == 1:
    US_img = image_polar(US_img, xdc,field,param)# polar convert the image
    image_plot_curvelinear(np.fliplr(US_img),
                        img_length = field["axial_size"]*rf_data.shape[1]+(xdc["radius"]*(1-np.cos(xdc["angle"]/2))), # length of image, [m]
                        fig_title='US B Scan')
else:
    image_plot_linear(np.fliplr(US_img),
                  img_length = field["axial_size"]*rf_data.shape[1], # length of image, [m]
                  lateral_length = xdc['element_num']*xdc["element_width"], # width of image, [m]
                  fig_title='US B Scan')

In [ ]:
data_dir

In [ ]:
title = raw_rf_path.split('\\')[-1].split('.')[0]

In [ ]:
plt.savefig(f'{title}.png')